In [9]:
import sqlite3
from sqlite3 import Error

def connect(db_file):
    """ create a database connection to a SQLite database """
    conn = sqlite3.connect(db_file)
    return conn


In [164]:
DB_FILE = "/tmp/company_db.db"

In [165]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select 1, SQLITE_VERSION();")
    res = c.fetchone()
    print(res)

(1, '3.28.0')


In [174]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("DROP TABLE IF EXISTS Employee")
    c.execute("DROP TABLE IF EXISTS Task")
    res = c.fetchone()
    print(res)

None


In [175]:
create_table = '''
CREATE TABLE IF NOT EXISTS Employee (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name text NOT NULL,
  sallary money NOT NULL
)
'''

In [176]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(create_table)

In [177]:
insert_sql = '''
INSERT INTO Employee (name, sallary) VALUES (?,?)
''';

In [178]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(insert_sql, ("Jakub", 2500.00))
    c.execute(insert_sql, ("Michał", 3500.00))
    print(c.lastrowid)

2


In [179]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Employee")
    rows = c.fetchall() 
    for row in rows:
        print(row)


(1, 'Jakub', 2500)
(2, 'Michał', 3500)


In [180]:
create_tasks_table = """
    CREATE TABLE IF NOT EXISTS Task (
        id integer PRIMARY KEY,
        employee_id integer,
        name text NOT NULL,
        priority integer,
        FOREIGN KEY (employee_id) REFERENCES Employee(id)
);
"""

conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute(create_tasks_table)

In [182]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (10, "Learn DELETE FROM TABLE sql stmt", 100))

In [184]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 10, 'Learn DELETE FROM TABLE sql stmt', 100)


Note that Foreign keys constraints are not enabled by default in SQLite, you have to enable them first by the running the following command:

In [185]:
enable_fk_check = '''PRAGMA foreign_keys = ON;'''
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute(enable_fk_check)

In [186]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("PRAGMA foreign_keys")
    rows = c.fetchone() 
    for row in rows:
        print(row)

1


In [188]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (1, "Learn DELETE FROM TABLE sql stmt", 100))

In [190]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("Select * from Task")
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 10, 'Learn DELETE FROM TABLE sql stmt', 100)
(2, 1, 'Learn DELETE FROM TABLE sql stmt', 100)


In [191]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("PRAGMA foreign_keys = ON;")
    c.execute("INSERT INTO Task (employee_id, name, priority) VALUES (?, ?, ?)", (100, "Learn DELETE FROM TABLE sql stmt", 100))

IntegrityError: FOREIGN KEY constraint failed

In [193]:
conn = connect(DB_FILE)
with conn:
    c = conn.cursor()
    c.execute("""
        Select * from Employee e 
        JOIN Task t on e.id = t.employee_id;
    """)
    rows = c.fetchall() 
    for row in rows:
        print(row)

(1, 'Jakub', 2500, 2, 1, 'Learn DELETE FROM TABLE sql stmt', 100)
